<a href="https://colab.research.google.com/github/Neilus03/ROADSIGN_ONTOLOGY/blob/main/Ontology_population_NCS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install -U owlready2 rdflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.5 MB/s eta 0:00:00
  Created wheel for owlready2: filename=Owlready2-0.41-cp310-cp310-linux_x86_64.whl size=24170167 sha256=1ef9239a03fab8beaa754f6d7ad094bd5a77a9f2b50d58fbe97a4fda81e7c37a
  Stored in directory: /root/.cache/pip/wheels/da/02/c0/a65e08daab6da8123998823c9aee7bba656b1500ab05d183ef
Successfully built owlready2


In [10]:
import rdflib
from owlready2 import get_ontology
from rdflib import Graph
from rdflib.namespace import RDF, FOAF
from rdflib import Namespace
from rdflib import Literal, XSD

In [11]:
import urllib.request

url = 'http://www.iiia.csic.es/~marco/kr/roadsign-data.ttl'
filename = 'roadsign-data.ttl'

urllib.request.urlretrieve(url, filename)


('roadsign-data.ttl', <http.client.HTTPMessage at 0x7ff3d0ca49d0>)

In [12]:
with open('roadsign-data.ttl', 'r') as f:
    data = f.read()

# Replace the colour terms
data = data.replace('red', 'NCS 1080-R')
data = data.replace('blue', 'NCS 4055-R95B')
data = data.replace('white', 'NCS S0500-N')
data = data.replace('black', 'NCS S9000-N')

# Write the modified TTL file
with open('modified.ttl', 'w') as f:
    f.write(data)

In [16]:


#Information in the web of roadsings
graph  = Graph()
graph.parse("/content/modified.ttl")

#Ontologi to populate (Our ontologi)
Graph_onto = Graph()
onto = get_ontology("/content/Population_Session8").load()

# Nueva sección

In [17]:
RSS = Namespace("http://www.iiia.csic.es/~marco/kr/roadsign-schema#")
instances = {}
for s, p, o in graph:   #graph is a list of tuples, each tuple is called a Triplet of the form (subject, predicate, object)
 if p == RDF.type and o == RSS.road_sign:  #If the predicate of a triplet is of type RDF and the object is a RoadSign
     instances[s] = onto.RoadSign()

In [18]:
instances

{rdflib.term.BNode('nbdbad89ce06d48d0a3897287d7bbc05eb12'): Population_Session8.roadsign178,
 rdflib.term.BNode('nbdbad89ce06d48d0a3897287d7bbc05eb25'): Population_Session8.roadsign179,
 rdflib.term.BNode('nbdbad89ce06d48d0a3897287d7bbc05eb59'): Population_Session8.roadsign180,
 rdflib.term.BNode('nbdbad89ce06d48d0a3897287d7bbc05eb21'): Population_Session8.roadsign181,
 rdflib.term.BNode('nbdbad89ce06d48d0a3897287d7bbc05eb51'): Population_Session8.roadsign182,
 rdflib.term.BNode('nbdbad89ce06d48d0a3897287d7bbc05eb41'): Population_Session8.roadsign183,
 rdflib.term.BNode('nbdbad89ce06d48d0a3897287d7bbc05eb46'): Population_Session8.roadsign184,
 rdflib.term.BNode('nbdbad89ce06d48d0a3897287d7bbc05eb10'): Population_Session8.roadsign185,
 rdflib.term.BNode('nbdbad89ce06d48d0a3897287d7bbc05eb6'): Population_Session8.roadsign186,
 rdflib.term.BNode('nbdbad89ce06d48d0a3897287d7bbc05eb29'): Population_Session8.roadsign187,
 rdflib.term.BNode('nbdbad89ce06d48d0a3897287d7bbc05eb52'): Population_

In [19]:
for s, p, o in graph:
  if p == RDF.type and o == RSS.road_sign:
    instances[s] = onto.RoadSign()

for s, p, o in graph:
  if p == FOAF.depicts:
    instances[o].filename = [s.toPython()]

  if s in instances:
    value = o.toPython().upper().replace(" ","_")
    value = value.replace("%", "_percent")
    value = value.replace(",", ".")
    
    if (p == RSS.shape):
      instances[s].shape = [onto.Shape(value)]

    if (p == RSS.ground_colour):
      if value.startswith("NCS "):
        instances[s].groundColor = [onto.NCSColor(value[4:])]
      else:
        instances[s].groundColor = [onto.Color(value)]

    if (p == RSS.border_colour):
      if value.startswith("NCS "):
        instances[s].borderColor = [onto.NCSColor(value[4:])]
      else:
        instances[s].borderColor = [onto.Color(value)]

    if (p == RSS.symbol):
      instances[s].symbol = [onto.Symbol(value)]

    if (p == RSS.symbol_colour):
      if value.startswith("NCS "):
        instances[s].symbolColor = [onto.NCSColor(value[4:])]
      else:
        instances[s].symbolColor = [onto.Color(value)]

    if (p == RSS.symbol_value):      
      instances[s].symbolValue = [onto.Value(value)]


AttributeError: ignored

In [ ]:
from owlready2 import *

In [ ]:
sync_reasoner()

In [ ]:
#What is the shape of ‘danger warning’ signs?
def Danger_Warning_Shape():
  shapes = set()
  for individual in onto.individuals(): 
    if isinstance(individual, onto.DangerWarningSign) and individual.shape is not None:
      shapes.add(str(individual.shape).replace("Population_Session8.", ""))
  return shapes

shapes = Danger_Warning_Shape()
shapes = ", ".join(list(shapes))
print(f"The shape of a Danger Warining Sign is: {shapes}")

In [ ]:
def RoadSings_Pedestrians():
  RoadSigns_pedestrians = []
  for individual in onto.individuals():
      # Check if the individual represents a RoadSign with a "PEDESTRIAN" symbol
      if isinstance(individual, onto.RoadSign) and individual.symbol is not None and "PEDESTRIAN" in str(individual.symbol):
          RoadSigns_pedestrians.append(individual)
  return RoadSigns_pedestrians
  
RoadSigns_pedestrians = RoadSings_Pedestrians()
print(f"The RoadSings with the a pedestrian symbol are: {RoadSigns_pedestrians}")

In [ ]:
#Which road signs are depicted in image IMG_20230202_154357.jpg?
def get_roadsign(image):
  roadsigns = []
  for x in onto.individuals():
    if isinstance(x, onto.RoadSign) and len(x.filename[0]) > 0 and x.filename[0] is not None:
      if image in x.filename[0]:
        roadsigns.append(x)
  return roadsigns

get_roadsign("IMG_20230202_154357.jpg")


In [ ]:
#Are there instances of a ‘compulsory minimum speed’ sign?

In [ ]:
#What percentage of road signs depicted in the image repository are 
#prohibitory signs?

rs = [x for x in onto.individuals() if isinstance(x,onto.RoadSign)]
ps = [x for x in onto.individuals() if isinstance(x,onto.ProhibitorySign)]
print(round(len(ps)/len(rs)*100,2),"%")

In [ ]:
for i in onto.individuals():
  print(i)

In [ ]:
#Is there any road sign in the image repository that informs about a restriction applicable to lorries or other large 
#vehicles?
def Restriction_large_vehicles():
  RoadSigns_pedestrians = []
  for individual in onto.individuals():
      # Check if the individual represents a RoadSign with a "PEDESTRIAN" symbol
      if isinstance(individual, onto.RoadSign) and individual.symbol is not None:
        if "" in str(individual.symbol):
          RoadSigns_pedestrians.append(individual)
  return RoadSigns_pedestrians
  
RoadSigns_pedestrians = RoadSings_Pedestrians()
print(f"The RoadSings with the a pedestrian symbol are: {RoadSigns_pedestrians}")

In [ ]:
#onto.save("/content/Population_Session8")